# Full Pipeline


The purpose of this notebook is to demonstrate the complete production pipeline for the patent novelty assessment system.

**Pipeline Setup:**
1. Load models and data
2. Process user input (patent title, abstract, claims)
3. Retrieve similar patents (FAISS + Online search)
4. Extract features for each patent pair
5. Score similarity using PyTorch Neural Network
6. Generate novelty assessment
7. Create LLM explanation

**Usage:** Run all cells sequentially to see the full pipeline in action.


In [1]:
import sys
from pathlib import Path
import numpy as np
import json
import os
from typing import Dict, List, Optional

# Get project root - handle both notebook dir and project root execution
current_path = Path().resolve()
if current_path.name == 'notebooks':
    project_root = current_path.parent
elif current_path.name == 'CS372-final-project':
    project_root = current_path
else:
    # Assume we're in project root
    project_root = current_path

sys.path.insert(0, str(project_root))
os.chdir(project_root)
print(f"Working directory: {os.getcwd()}")

# Set SerpAPI key directly
serpapi_key = "6e0db2eae6b21423feb180ae3a5b47063df9f9ff7d18eceaafff67ddae20770b"
os.environ['SERPAPI_KEY'] = serpapi_key
print(f"SerpAPI key configured ({len(serpapi_key)} characters)")

from src.app.patent_analyzer import PatentAnalyzer
from src.app.pytorch_classifier import PyTorchPatentClassifier
from src.features.feature_extract import FeatureExtractor
from src.embeddings.patent_sberta import PatentEmbedder

print("Imports successful")

Working directory: /Users/abhinavmeduri/projects/CS372-final-project
SerpAPI key configured (64 characters)


Imports successful


## Step 1: Initialize the Patent Analyzer

Load all required models and data.


In [2]:
# Use the hardcoded SerpAPI key
serpapi_key = "6e0db2eae6b21423feb180ae3a5b47063df9f9ff7d18eceaafff67ddae20770b"
analyzer = PatentAnalyzer(
    use_online_search=True,
    use_llm_keywords=True,
    serpapi_key=serpapi_key
)

analyzer.load()

print("Patent Analyzer initialized")
print(f"Embedding model: PatentSBERTa")
print(f"Classification model: PyTorch Neural Network")
print(f"LLM explainer: Phi-3 (Ollama)")
print(f"Online search: Enabled (SerpAPI)")
print(f"Patent database: {len(analyzer.patents)} patents")

  Loading Patent Analyzer resources...
  Loading embeddings...
  Loaded 200,000 embeddings
  Loading PatentSBERTa model...


/Users/abhinavmeduri/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: AI-Growth-Lab/PatentSBERTa


INFO:data.api.online_search:[OK] Using SerpAPI for Google Patents search (millions of patents)


INFO:data.api.online_search:  API key configured: 6e0db2ea...


INFO:src.app.pytorch_classifier:Initialized (device: mps)


INFO:src.app.pytorch_classifier:Loaded from models/pytorch_nn


  PatentSBERTa model loaded
  Initializing Phi-3 explainer...
Phi-3 Ollama Explainer initialized
  Model: phi3
  KV caching: Enabled (built-in)
  Metal acceleration: Enabled
  Phi-3 explainer ready
  Initializing LLM keyword extractor...
  LLM Keyword Extractor ready
  Initializing online search...
  Online Patent Search ready (SerpAPI)
  Loading PyTorch model...
  PyTorch model and feature extractor ready
  Ready! (Components loaded on-demand)
Patent Analyzer initialized
Embedding model: PatentSBERTa
Classification model: PyTorch Neural Network
LLM explainer: Phi-3 (Ollama)
Online search: Enabled (SerpAPI)
Patent database: 0 patents


## Step 2: Define Example Patent

Input a patent application to assess for novelty.

In [3]:
example_patent = {
    "title": "Real-time video recommendation system using sequence-aware user embeddings",
    "abstract": "A recommendation system for a video streaming platform that generates personalized suggestions by learning embeddings from users' watch histories. The system updates embeddings in real time and combines short-term and long-term viewing patterns to improve recommendation accuracy.",
    "claims": [
        "1. A video recommendation system comprising: a watch history tracker configured to log sequences of videos watched by a user; an embedding generator configured to produce low-dimensional embeddings representing a user's viewing preferences based on the sequence of watched videos; a model updater configured to adjust embeddings and predictive model parameters in real time as new videos are watched; and a recommendation engine configured to rank and suggest videos by combining user embeddings with video metadata features such as genre, duration, and popularity.",
        "2. The system of claim 1, wherein the embedding generator uses a recurrent neural network or transformer-based model to capture temporal patterns in user watch behavior.",
        "3. The system of claim 1, further comprising a short-term preference module that emphasizes recently watched videos to adjust immediate recommendations.",
        "4. The system of claim 1, wherein the recommendation engine filters suggested videos to prioritize content available in the user's region and compatible with their device type."
    ]
}

## Step 3: Run Novelty Assessment

Execute the complete pipeline.


In [4]:
input_text = f"{example_patent['title']}\n\n{example_patent['abstract']}\n\nClaims:\n" + "\n".join(example_patent['claims'])

result = analyzer.analyze(input_text)


INFO:data.api.online_search:Generated 5 search terms via LLM


Generated 5 search terms: ['real-time video recommendation AND user embedding patterns', 'sequence analysis in streaming platforms OR viewing pattern recognition', 'personalized content suggestion system AND embeddings update mechanism']...


INFO:data.api.online_search:Searching 5 terms with max_per_term=10


INFO:data.api.online_search:Using SerpAPI: True, API key present: True


INFO:data.api.online_search:[1/5] Searching term: 'real-time video recommendation AND user embedding patterns...'


...

INFO:data.api.online_search:    First result: JP7213206B2 - intelligent auto assistant...


INFO:data.api.online_search:Total online patents found: 50 across 5 terms


search_multiple_terms returned 5 terms with results
Term 'real-time video recommendation AND user embedding ...': 10 results (type: <class 'list'>)
Term 'sequence analysis in streaming platforms OR viewin...': 10 results (type: <class 'list'>)
Term 'personalized content suggestion system AND embeddi...': 10 results (type: <class 'list'>)
Term '(video watch history) AND (recommendation accuracy...': 10 results (type: <class 'list'>)
Term 'short-term and long-term viewing patterns AND real...': 10 results (type: <class 'list'>)
Online search found 48 unique patents across 5 terms


Ranking-based assessment: scored 20 patents, mean similarity=0.730, novelty=0.270, percentile=0.0%
Generating explanation via Ollama (with KV caching)...


[OK] Generated 1114 tokens in 44.5s (25.0 tok/s)


## Step 4: Display Results

Show the novelty assessment results.


In [5]:
input_text = f"""Title: {example_patent["title"]}

Abstract: {example_patent["abstract"]}

Claims:
{chr(10).join(example_patent["claims"])}
"""

result = analyzer.analyze(input_text)

# Display results in frontend-like format
print("Patent Novelty Assessment Results")

print(f"\nNovelty Score: {result.novelty_score:.3f}")
if result.novelty_score > 0.5:
    print(f"   Likely Novel")
else:
    print(f"   Potential Prior Art Found")

if result.similar_patents:
    print(f"\nFound {len(result.similar_patents)} Similar Patents:")
    for idx, patent in enumerate(result.similar_patents[:5], 1):
        print(f"\n   {idx}. Patent ID: {patent.get('patent_id', 'N/A')}")
        print(f"      Similarity: {patent.get('similarity_score', 0):.3f}")
        if 'title' in patent:
            print(f"      Title: {patent['title'][:60]}...")
        if 'year' in patent:
            print(f"      Year: {patent['year']}")

if result.explanation:
    print(f"\nExplanation:")
    explanation_lines = result.explanation.split('\n')
    for line in explanation_lines[:5]:  # Show first 5 lines
        print(f"   {line}")
    if len(explanation_lines) > 5:
        print(f"   ... ({len(explanation_lines) - 5} more lines)")



INFO:data.api.online_search:Generated 5 search terms via LLM


Generated 5 search terms: ['real-time learning embeddings AND video streaming recommendations', 'personalized suggestions system OR embedding generation for viewing patterns', '(short term) AND (long term) AND (viewing history) AND (recommendation accuracy)']...


INFO:data.api.online_search:Searching 5 terms with max_per_term=10


INFO:data.api.online_search:Using SerpAPI: True, API key present: True


INFO:data.api.online_search:[1/5] Searching term: 'real-time learning embeddings AND video streaming recommendations...'


...

INFO:data.api.online_search:    First result: US9064010B2 - Encoding and transcoding for mobile media...


INFO:data.api.online_search:Total online patents found: 50 across 5 terms


search_multiple_terms returned 5 terms with results
Term 'real-time learning embeddings AND video streaming ...': 10 results (type: <class 'list'>)
Term 'personalized suggestions system OR embedding gener...': 10 results (type: <class 'list'>)
Term '(short term) AND (long term) AND (viewing history)...': 10 results (type: <class 'list'>)
Term 'real-time embedding updates AND user watch histori...': 10 results (type: <class 'list'>)
Term 'adaptive recommendation algorithms AND video conte...': 10 results (type: <class 'list'>)
Online search found 48 unique patents across 5 terms


Ranking-based assessment: scored 20 patents, mean similarity=0.662, novelty=0.338, percentile=0.0%
Generating explanation via Ollama (with KV caching)...


[OK] Generated 854 tokens in 36.6s (23.3 tok/s)
Patent Novelty Assessment Results

Novelty Score: 0.338
   Potential Prior Art Found

Found 58 Similar Patents:

   1. Patent ID: 12288236
      Similarity: 0.000
      Title: This invention relates generally to recommendation systems, ...
      Year: 2025

   2. Patent ID: 11055629
      Similarity: 0.000
      Title: This invention relates generally to embedding models, and mo...
      Year: 2021

   3. Patent ID: US20240386015A1
      Similarity: 0.000
      Title: Composite symbolic and non-symbolic artificial intelligence ...
      Year: 2024

   4. Patent ID: US12223591B2
      Similarity: 0.000
      Title: Cross reality system for large scale environment reconstruct...
      Year: 2025

   5. Patent ID: CN113544659B
      Similarity: 0.000
      Title: Computer-implemented method, device, and computer-readable s...
      Year: 2025

Explanation:
   ## EXECUTIVE SUMMARY
   Upon review of the patent application for a real-time video

## Step 5: Pipeline Breakdown

Examine each step of the pipeline in detail.


In [6]:
# Format input text
input_text = f"""Title: {example_patent["title"]}

Abstract: {example_patent["abstract"]}

Claims:
{chr(10).join(example_patent["claims"])}
"""

# Run the complete pipeline
result = analyzer.analyze(input_text)

print("Patent Novelty Assessment Results")

novelty_score = result.novelty_score if result.novelty_score is not None else 0.5
rank_percentile = result.search_metadata.get('rank_percentile', None) if result.search_metadata else None
top_k_scored = result.search_metadata.get('top_k_scored', 0) if result.search_metadata else 0

print(f"\nNovelty Score: {novelty_score:.3f}")
if rank_percentile is not None:
    print(f"Rank Percentile: {rank_percentile:.1f}%")
    print(f"Interpretation: Ranks in top {rank_percentile:.1f}% most novel among {top_k_scored} analyzed patents")
else:
    print(f"Interpretation: Continuous score (0.0 = Not Novel, 1.0 = Highly Novel)")

print(f"\nScale: 0.0 = Not Novel (high similarity) | 1.0 = Highly Novel (low similarity)")
print(f"Current Score: {novelty_score:.3f} indicates {'high' if novelty_score > 0.7 else 'moderate' if novelty_score > 0.5 else 'low'} novelty")

if result.similar_patents:
    print(f"\nTop {min(10, len(result.similar_patents))} Ranked Similar Patents:")
    
    sorted_patents = sorted(
        result.similar_patents[:10],
        key=lambda p: p.get('model_novelty', 1 - p.get('similarity', 0)),
        reverse=True
    )
    
    for idx, patent in enumerate(sorted_patents, 1):
        rank = patent.get('rank', idx)
        model_sim = patent.get('model_similarity', patent.get('similarity', 0))
        model_novelty = 1 - model_sim if model_sim else (1 - patent.get('similarity', 0))
        
        print(f"\n#{rank} - Patent ID: {patent.get('patent_id', 'N/A')}")
        print(f"   Model Similarity: {model_sim:.3f} | Model Novelty: {model_novelty:.3f}")
        if 'title' in patent:
            print(f"   Title: {patent['title'][:70]}...")
        if 'year' in patent:
            print(f"   Year: {patent['year']}")
        if 'source' in patent:
            print(f"   Source: {patent['source']}")

if result.explanation:
    print(f"\n\nDetailed Explanation:")
    explanation_lines = result.explanation.split('\n')
    for line in explanation_lines[:8]:
        print(f"   {line}")
    if len(explanation_lines) > 8:
        print(f"   ... ({len(explanation_lines) - 8} more lines)")



INFO:data.api.online_search:Generated 5 search terms via LLM


Generated 5 search terms: ['recommendation system AND personalized suggestions OR video streaming platform', '(embeddings learning) AND (watch histories) AND (real-time update)', 'short-term viewing patterns AND long-term embeddings']...


INFO:data.api.online_search:Searching 5 terms with max_per_term=10


INFO:data.api.online_search:Using SerpAPI: True, API key present: True


INFO:data.api.online_search:[1/5] Searching term: 'recommendation system AND personalized suggestions OR video streaming platform...'


...

INFO:data.api.online_search:    First result: CN109195020B - An AR-enhanced game live broadcast method and syst...


INFO:data.api.online_search:Total online patents found: 50 across 5 terms


search_multiple_terms returned 5 terms with results
Term 'recommendation system AND personalized suggestions...': 10 results (type: <class 'list'>)
Term '(embeddings learning) AND (watch histories) AND (r...': 10 results (type: <class 'list'>)
Term 'short-term viewing patterns AND long-term embeddin...': 10 results (type: <class 'list'>)
Term 'recommendation accuracy improvement AND sensor err...': 10 results (type: <class 'list'>)
Term '(video streaming platform) AND (user watch behavio...': 10 results (type: <class 'list'>)
Online search found 50 unique patents across 5 terms


Ranking-based assessment: scored 20 patents, mean similarity=0.672, novelty=0.328, percentile=0.0%
Generating explanation via Ollama (with KV caching)...


[OK] Generated 975 tokens in 38.0s (25.7 tok/s)
Patent Novelty Assessment Results

Novelty Score: 0.328
Rank Percentile: 0.0%
Interpretation: Ranks in top 0.0% most novel among 20 analyzed patents

Scale: 0.0 = Not Novel (high similarity) | 1.0 = Highly Novel (low similarity)
Current Score: 0.328 indicates low novelty

Top 10 Ranked Similar Patents:

#1 - Patent ID: 12288236
   Model Similarity: 0.000 | Model Novelty: 1.000
   Title: This invention relates generally to recommendation systems, and partic...
   Year: 2025
   Source: local

#2 - Patent ID: 11055629
   Model Similarity: 0.004 | Model Novelty: 0.996
   Title: This invention relates generally to embedding models, and more particu...
   Year: 2021
   Source: local

#3 - Patent ID: US11675836B2
   Model Similarity: 0.680 | Model Novelty: 0.320
   Title: Mobile media pause and resume...
   Year: 2023
   Source: online

#4 - Patent ID: US11685048B2
   Model Similarity: 0.680 | Model Novelty: 0.320
   Title: Enhancing robot learn

## Summary

1. **Input Processing**: Parse patent title, abstract, and claims
2. **Retrieval**: Find similar patents using FAISS (local) and optionally online search
3. **Feature Extraction**: Compute 10 features for each patent pair (reduced from 13 via ablation study)
4. **Classification**: Score similarity using PyTorch Neural Network (91.73% accuracy, architecture: [256, 128])
5. **Ranking-Based Assessment**: Score top-K candidates (top 20) and derive novelty from rank distribution
6. **Explanation**: Generate detailed explanation using Phi-3 LLM

**Key Components:**
- PatentSBERTa embeddings for semantic similarity
- PyTorch Neural Network for classification
- Phi-3 (Ollama) for natural language explanations
- Feature engineering (10 features: cosine similarity, embeddings, text overlap, metadata)

**Output:**
- Novelty score (0-1, continuous scale, higher = more novel)
- Rank percentile (how novel compared to top-K analyzed patents)
- Ranked list of similar prior art patents with model similarity/novelty scores
- Detailed explanation of the assessment
